Surprise 패키지 다운로드

In [ ]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 9.2 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633728 sha256=4e0ad9693e7c07e675fe9a19faf3a2f284c68a9109750e39c36da66faa5353b2
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
import pandas as pd

products = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CP1/products.csv')
orders = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CP1/orders.csv')
order_products_prior = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CP1/order_products__prior.csv')

print('products shape :', products.shape)
print('orders shape :', orders.shape)
print('order_products_prior shape :', order_products_prior.shape)

products shape : (49688, 4)
orders shape : (3421083, 7)
order_products_prior shape : (32434489, 4)


Surprise 패키지는 추천시스템 구현을 도와주는 파이썬 패키지로서 'userID', 'itemID', 'rating'이라는 세가지의 정해진 칼럼을 기반으로 추천시스템 모델을 만들어준다. 그렇기에 우리의 데이터에서 rating 즉, 유저의 상품 선호도를 알 수 있는 지표는 '재구매율'로 판단할 수 있기 때문에 '재구매율'을 기반으로 데이터를 맞추어준다.

In [ ]:
prior = pd.merge(orders, order_products_prior, on = 'order_id', how = 'inner')
up_reorder = prior.groupby(['user_id', 'product_id'])['reordered'].mean().to_frame('up_reorder').reset_index()
up_reorder.head()

user_id  product_id  up_reorder
0        1         196    0.900000
1        1       10258    0.888889
2        1       10326    0.000000
3        1       12427    0.900000
4        1       13032    0.666667

In [ ]:
up_reorder['up_reorder'] = round(up_reorder['up_reorder']*5)
up_reorder.head()

user_id  product_id  up_reorder
0        1         196         4.0
1        1       10258         4.0
2        1       10326         0.0
3        1       12427         4.0
4        1       13032         3.0

0~5점사이의 평점지표와 유사하게 up_reorder칼럼을 스케일링을 해주었지만 0점의 데이터가 너무나도 많고 0점의 데이터는 모델 설계에 있어서 critical한 데이터가 아니기 때문에 모두 제거하여 데이터를 축소시켜준다.

In [ ]:
up_reorder['up_reorder'].value_counts()

0.0    7982695
2.0    2204600
4.0    1682690
3.0    1029149
5.0     408819
Name: up_reorder, dtype: int64

In [ ]:
up_reorder = up_reorder[up_reorder['up_reorder']>0]

In [ ]:
up_reorder

user_id  product_id  up_reorder
0               1         196         4.0
1               1       10258         4.0
3               1       12427         4.0
4               1       13032         3.0
5               1       13176         2.0
...           ...         ...         ...
13307944   206209       40534         2.0
13307945   206209       40992         3.0
13307946   206209       41213         4.0
13307948   206209       43961         3.0
13307952   206209       48742         2.0

[5325258 rows x 3 columns]

In [ ]:
# up_reorder.to_csv('up_reorder.csv', index=False)

추천시스템 모델에는 Content based, Collaborative filtering, Deeplearning이 있고 이번 모델에서는 Collaborative filtering 그 중에서도 Memory-Based Approach인 KNN, Cosine Similarity, Pearson Correlation이 아닌 Model-Based Approach인 행렬분해(MF, Matrix Factorization)로 접근하고자 한다. 그 중에서도 현업에서 가장 자주 쓰이고 Sparse한 데이터에도 높은 성능과 넓을 확장성을 제공해주는 SVD알고리즘을 적용해보았다.

In [ ]:
from surprise import Reader, Dataset, accuracy, SVD
from surprise.model_selection import train_test_split, GridSearchCV

param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200]}
reader = Reader(rating_scale=(2.0, 5.0))
data = Dataset.load_from_df(up_reorder[['user_id', 'product_id', 'up_reorder']], reader=reader)
train, test = train_test_split(data, test_size=0.2, random_state=42)
 
algo = SVD(random_state=42)
algo.fit(train)
prediction = algo.test(test)
accuracy.rmse(prediction)

RMSE: 0.9826


0.9826069934099015

uid = 유저아이디

iid = 아이템아이디

r_ui = 실제 평점

est = 예측 평점

In [ ]:
prediction[5:10]

[Prediction(uid=170773, iid=27156, r_ui=2.0, est=2.0597193262992146, details={'was_impossible': False}),
 Prediction(uid=149079, iid=20591, r_ui=4.0, est=2.8577213493599016, details={'was_impossible': False}),
 Prediction(uid=136489, iid=43251, r_ui=4.0, est=3.0905995453044475, details={'was_impossible': False}),
 Prediction(uid=34985, iid=19863, r_ui=2.0, est=2.6312651609271898, details={'was_impossible': False}),
 Prediction(uid=138008, iid=40612, r_ui=5.0, est=2.9419057872520815, details={'was_impossible': False})]

In [ ]:
result = [(pred.uid, pred.iid, pred.est) for pred in prediction[:5]]
result

[(165002, 11000, 3.1806295574557693),
 (25871, 27104, 2.9584209200648464),
 (152364, 48322, 3.0546736259101115),
 (139422, 45310, 2.720403730627238),
 (1990, 27390, 2.554902289476904)]

학습에 시간이 좀 걸려서 한번 돌리면 피클파일로 저장/불러오기로 했다.

In [ ]:
import pickle

with open("algo.pickle","wb") as fw:
  pickle.dump(algo, fw)

유저별로 재구매율, 즉 평점이 4점 이상인 상품들만 따로 모아서 리스트로 만들어줌으로써 유저별로 가장 선호하는(재구매를 많이하는) 상품 리스트를 알 수 있는 테이블을 만들었다.

In [ ]:
up_reorder = up_reorder[up_reorder['up_reorder']>=4].groupby('user_id')['product_id'].apply(list).reset_index()

In [ ]:
up_reorder

user_id                                         product_id
0             1                         [196, 10258, 12427, 25133]
1             2  [1559, 2002, 12000, 16589, 18523, 19051, 19156...
2             3   [9387, 17668, 18599, 21903, 39190, 43961, 47766]
3             5                                     [11777, 26604]
4             7  [4920, 6361, 13198, 17638, 21137, 27690, 30391...
...         ...                                                ...
153470   206203                              [15693, 21469, 37188]
153471   206206  [11520, 11954, 13045, 23737, 27086, 38530, 401...
153472   206207  [2954, 3397, 3469, 11068, 11140, 13176, 19678,...
153473   206208  [365, 1463, 1804, 2643, 5876, 6297, 9339, 1152...
153474   206209  [6846, 9405, 13176, 19348, 23594, 24852, 38167...

[153475 rows x 2 columns]

In [ ]:
up_reorder.to_csv('up_reorder2.csv', index=False)

위의 리스트를 통해 얻은 최애 상품들 중 해당 상품이 속하는 소분류, 대분류가 둘 다 일치하는 카테고리의 모든 상품들에 대해 해당 유저가 한번도 주문한 적은 없지만 SVD모델을 통해 만약 사용했을 경우 재구매율(평점)이 얼마나 나오는지 예측하고 나온 예측값의 상위 3개의 상품을 추천해주는 함수를 정의하였다.

협업필터링의 핵심은 결국, 내가 구매하지 않은 상품이지만 다른 사람들이 높게 평가하는(재구매율이 높은 혹은 평점이 높은) 상품이면 사람인 이상 나도 사용해본다면 좋은 평가 혹은 만족을 할 가능성이 높다는 것이다. 즉, 우리 모델에서는 같은 카테고리인 상품(소분류, 대분류가 동일한)에 한해서 내가 주문한적 없는 상품이지만 주문했을 경우의 평점(재구매율)을 예측해보고 그것이 높다면 추천해주는 방식인 것이다.

In [ ]:
def sortkey_est(pred):
  return pred.est

def find_name(arg):
  return products[products['product_id']==arg]['product_name'].values[0]

def recommend(user_id):
  dic = {}
  try:
    a = up_reorder[up_reorder['user_id']==user_id]['product_id'].values[0]
  except:
    return 'No recommend'
  # k = 2
  for i in a:
    aisle = products[products['product_id']==i]['aisle_id'].values[0]
    department = products[products['product_id']==i]['department_id'].values[0]
    candi = products[(products['aisle_id']==aisle) & (products['department_id']==department)]['product_id'].to_list()
    predi = [algo.predict(str(user_id), str(j)) for j in candi]
    predi.sort(key=sortkey_est, reverse=True)
    top_predi = predi[:3]
    top_product_id = [int(pred.iid) for pred in top_predi]
    top_product_name = products[products.product_id.isin(top_product_id)]['product_name'].to_list()
    # if find_name(aisle)+'/'+find_name(department) in dic:
    #   dic[find_name(aisle)+'/'+find_name(department)+str(k)] = top_product_name
    #   k += 1
    # else:
    #   dic[find_name(aisle)+'/'+find_name(department)] = top_product_name
    dic[find_name(aisle)+'/'+find_name(department)] = top_product_name
  return dic

user_id=1인 유저의 추천리스트로서 key값은 위에서 구한 이 유저의 최애 리스트 상품들에 해당하는 소분류/대분류이며 value값은 그 상품과 동일한 카테고리 영역에서 내가 사용했을 경우, 재구매율이 클 상품들을 예측한 상품들 중 상위 3개의 상품들이다.

In [ ]:
recommend(1)

{'Coconut Chocolate Chip Energy Bar/Pure Coconut Water With Orange': ['Jasmine Tea Unfiltered Ginger Ale',
  'Sparkling Blush Grape Juice',
  'Soda'],
 'Organic Turkey Burgers/Gluten Free Quinoa Three Cheese & Mushroom Blend': ['Healthy Pop Butter Popcorn',
  'Uncured Cracked Pepper Beef',
  'Movie Theater Butter Pop Corn'],
 'Petit Suisse Fruit/Gluten Free Quinoa Three Cheese & Mushroom Blend': ['Dried Sweetened Cranberries',
  'Dry Roasted Pistachios',
  'Ground Flaxseed With Mixed Berries'],
 'Small & Medium Dental Dog Treats/Mint Chocolate Flavored Syrup': ['Sharp Cheddar',
  'Extra Sharp Cheddar Cheese',
  'Shaved Parmesan Cheese']}

In [ ]:
pd.DataFrame(recommend(1))

Coconut Chocolate Chip Energy Bar/Pure Coconut Water With Orange  \
0                  Jasmine Tea Unfiltered Ginger Ale                 
1                        Sparkling Blush Grape Juice                 
2                                               Soda                 

  Petit Suisse Fruit/Gluten Free Quinoa Three Cheese & Mushroom Blend  \
0                        Dried Sweetened Cranberries                    
1                             Dry Roasted Pistachios                    
2                 Ground Flaxseed With Mixed Berries                    

  Organic Turkey Burgers/Gluten Free Quinoa Three Cheese & Mushroom Blend  \
0                         Healthy Pop Butter Popcorn                        
1                        Uncured Cracked Pepper Beef                        
2                      Movie Theater Butter Pop Corn                        

  Small & Medium Dental Dog Treats/Mint Chocolate Flavored Syrup  
0                                      Sharp Cheddar              
1                         Extra Sharp Cheddar Cheese              
2                             Shaved Parmesan Cheese

실제로 위의 추천리스트와 아래의 해당되는 1번 유저의 최애 리스트를 비교해보면 어느 정도 유사하다는 것을 알 수가 있다.

In [ ]:
a= up_reorder[up_reorder['user_id']==1]['product_id'].values[0]
a

[196, 10258, 12427, 25133]

In [ ]:
b = []
for s in a:
  b.append(find_name(s))
b

['Soda', 'Pistachios', 'Original Beef Jerky', 'Organic String Cheese']

user_id=2 유저의 최애리스트 길이

In [ ]:
len(up_reorder[up_reorder['user_id']==2]['product_id'].values[0])

14

상품별로 소분류/대분류에 맞춰 추천리스트를 주게 되면 상품의 카테고리들이 겹치는 일이 발생하게 되는데 그에 맞춰서 추천리스트를 확인해보면 추천리스틀이 중복된다는 사실을 알 수가 있다. 그래서 중복허용을 금지해야 된다는 결론에 도달할 수 있다. 아래 예시는 칼럼수가 위의 결과값과 같이 14인 것을 알 수 있다.

In [ ]:
pd.DataFrame(recommend(2))

Cauliflower Florettes/Mint Chocolate Flavored Syrup  \
0                  Light Strawberry Blueberry Yogurt    
1                      Yogurt Strawberry Pomegranate    
2       Non-Fat Blueberry on the Bottom Greek Yogurt    

         Biotin 1000 mcg/Chocolate Sandwich Cookies  \
0  Vegan Santa Fe Style Flour Tortilla Rolled Tacos   
1                       Veggie-Style Mini Corn Dogs   
2                                      Street Tacos   

  Organic Honeycrisp Apples/Saline Nasal Mist  \
0                               Honey Mustard   
1                    Tikka Masala Curry Sauce   
2        Sweet & Smokey Kansas City Bbq Sauce   

  Mild Salsa Divino/Gluten Free Quinoa Three Cheese & Mushroom Blend  \
0                      Nacho Cheese White Bean Chips                   
1                        Chips Onion Chipotle Garlic                   
2  Restaurant Style Organic Chia & Quinoa Tortill...                   

  Cauliflower Florettes/Mint Chocolate Flavored Syrup2  \
0                  Light Strawberry Blueberry Yogurt     
1                      Yogurt Strawberry Pomegranate     
2       Non-Fat Blueberry on the Bottom Greek Yogurt     

  Mild Salsa Divino/Gluten Free Quinoa Three Cheese & Mushroom Blend3  \
0                      Nacho Cheese White Bean Chips                    
1                        Chips Onion Chipotle Garlic                    
2  Restaurant Style Organic Chia & Quinoa Tortill...                    

  Cauliflower Florettes/Mint Chocolate Flavored Syrup4  \
0                  Light Strawberry Blueberry Yogurt     
1                      Yogurt Strawberry Pomegranate     
2       Non-Fat Blueberry on the Bottom Greek Yogurt     

  Cauliflower Florettes/Mint Chocolate Flavored Syrup5  \
0                  Light Strawberry Blueberry Yogurt     
1                      Yogurt Strawberry Pomegranate     
2       Non-Fat Blueberry on the Bottom Greek Yogurt     

  Tri-Vi-Sol® Vitamins A-C-and D Supplement Drops for Infants/Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce  \
0                                     Organic Lemons                                                                              
1                                         Nectarines                                                                              
2                                         Cantaloupe                                                                              

  Jelly, Blackberry/Pomegranate Cranberry & Aloe Vera Enrich Drink  \
0                           Organic Ranch Veggie Dip                 
1                                 Mild El Nino Salsa                 
2                            Tzatziki Cucumber Sauce                 

  Organic Turkey Burgers/Gluten Free Quinoa Three Cheese & Mushroom Blend  \
0                         Healthy Pop Butter Popcorn                        
1                        Uncured Cracked Pepper Beef                        
2                      Movie Theater Butter Pop Corn                        

  Cauliflower Florettes/Mint Chocolate Flavored Syrup6  \
0                  Light Strawberry Blueberry Yogurt     
1                      Yogurt Strawberry Pomegranate     
2       Non-Fat Blueberry on the Bottom Greek Yogurt     

  Tri-Vi-Sol® Vitamins A-C-and D Supplement Drops for Infants/Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce7  \
0                                     Organic Lemons                                                                               
1                                         Nectarines                                                                               
2                                         Cantaloupe                                                                               

  Tri-Vi-Sol® Vitamins A-C-and D Supplement Drops for Infants/Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce8  
0                                     Organic Lemons              

아래는 중복을 허용X

In [ ]:
# 중복 허용 X
pd.DataFrame(recommend(2))

Cauliflower Florettes/Mint Chocolate Flavored Syrup  \
0                  Light Strawberry Blueberry Yogurt    
1                      Yogurt Strawberry Pomegranate    
2       Non-Fat Blueberry on the Bottom Greek Yogurt    

         Biotin 1000 mcg/Chocolate Sandwich Cookies  \
0  Vegan Santa Fe Style Flour Tortilla Rolled Tacos   
1                       Veggie-Style Mini Corn Dogs   
2                                      Street Tacos   

  Organic Honeycrisp Apples/Saline Nasal Mist  \
0                               Honey Mustard   
1                    Tikka Masala Curry Sauce   
2        Sweet & Smokey Kansas City Bbq Sauce   

  Mild Salsa Divino/Gluten Free Quinoa Three Cheese & Mushroom Blend  \
0                      Nacho Cheese White Bean Chips                   
1                        Chips Onion Chipotle Garlic                   
2  Restaurant Style Organic Chia & Quinoa Tortill...                   

  Tri-Vi-Sol® Vitamins A-C-and D Supplement Drops for Infants/Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce  \
0                                     Organic Lemons                                                                              
1                                         Nectarines                                                                              
2                                         Cantaloupe                                                                              

  Jelly, Blackberry/Pomegranate Cranberry & Aloe Vera Enrich Drink  \
0                           Organic Ranch Veggie Dip                 
1                                 Mild El Nino Salsa                 
2                            Tzatziki Cucumber Sauce                 

  Organic Turkey Burgers/Gluten Free Quinoa Three Cheese & Mushroom Blend  
0                         Healthy Pop Butter Popcorn                       
1                        Uncured Cracked Pepper Beef                       
2                      Movie Theater Butter Pop Corn

없는 user_id의 경우는 위에서 재구매율 4점 이상을 넘기는 상품들을 추려내면서 삭제된 경우이므로 추천리스트가 없다고 표시해준다.

In [ ]:
recommend(4)

'No recommend'